# Herramientas para procesamiento de texto (parte 2)

In [1]:
# Solo si se usa Google Colab
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ejemplo_features2').getOrCreate()

In [3]:
df = spark.createDataFrame([
        (0, 'Hola Spark conozco sobre Spark'),
        (0, 'Spark tiene varias varias varias herramientas'),
        (1, 'Se analiza datos con herramientas')],
        ['label', 'oracion'])

df.show(truncate=False)

+-----+---------------------------------------------+
|label|oracion                                      |
+-----+---------------------------------------------+
|0    |Hola Spark conozco sobre Spark               |
|0    |Spark tiene varias varias varias herramientas|
|1    |Se analiza datos con herramientas            |
+-----+---------------------------------------------+



In [4]:
# Convertir a palabras ("tokens")
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol='oracion', outputCol='palabras')
df2 = tokenizer.transform(df)

df2.show(truncate=False)

+-----+---------------------------------------------+----------------------------------------------------+
|label|oracion                                      |palabras                                            |
+-----+---------------------------------------------+----------------------------------------------------+
|0    |Hola Spark conozco sobre Spark               |[hola, spark, conozco, sobre, spark]                |
|0    |Spark tiene varias varias varias herramientas|[spark, tiene, varias, varias, varias, herramientas]|
|1    |Se analiza datos con herramientas            |[se, analiza, datos, con, herramientas]             |
+-----+---------------------------------------------+----------------------------------------------------+



### TF-IDF

Significa "Term Frequency - Inverse Document Frequency". Es un método que refleja la importancia de un término de un documento en un conjunto de documentos. Dado un término $t$, un documento $d$, y un número total de documentos $N$ (número total de documentos), el TF-IDF del término $t$, representado como $f_{t,d,D}$ será
$$ f_{t, d, N} = f_{t, d} ~log\left(\frac{N}{n_d}\right) $$
donde 
$f_{t, d}$ representa la frecuencia de aparición del término $t$ en el documento $d$ (número de veces que aparece el término $t$, dividido entre el número total de términos en $d$), $N$ el número total de documentos, y $n_d$ el número de documentos que contienen el término $t$.


In [5]:
# TF: Term frequency (alternativa 1: usando hashing)
from pyspark.ml.feature import HashingTF

# Bag of Words ("diccionario de palabras")
hasingtf = HashingTF(inputCol='palabras', outputCol='rawFeatures')
df3 = hasingtf.transform(df2)

df3.show(truncate=False)

+-----+---------------------------------------------+----------------------------------------------------+-------------------------------------------------------------------+
|label|oracion                                      |palabras                                            |rawFeatures                                                        |
+-----+---------------------------------------------+----------------------------------------------------+-------------------------------------------------------------------+
|0    |Hola Spark conozco sobre Spark               |[hola, spark, conozco, sobre, spark]                |(262144,[26632,68822,173558,258035],[1.0,1.0,2.0,1.0])             |
|0    |Spark tiene varias varias varias herramientas|[spark, tiene, varias, varias, varias, herramientas]|(262144,[82457,173047,173558,256945],[3.0,1.0,1.0,1.0])            |
|1    |Se analiza datos con herramientas            |[se, analiza, datos, con, herramientas]             |(262144,[101152,135

In [6]:
# TF: Term Frequency (alternativa 2)
from pyspark.ml.feature import CountVectorizer

# min DF: mínimo número de veces que debe aparecer una palabra
cvectorizer = CountVectorizer(inputCol='palabras', outputCol='rawFeatures', minDF=1)

df4 = cvectorizer.fit(df2).transform(df2)

df4.show(truncate=False)

+-----+---------------------------------------------+----------------------------------------------------+---------------------------------------+
|label|oracion                                      |palabras                                            |rawFeatures                            |
+-----+---------------------------------------------+----------------------------------------------------+---------------------------------------+
|0    |Hola Spark conozco sobre Spark               |[hola, spark, conozco, sobre, spark]                |(11,[0,5,7,9],[2.0,1.0,1.0,1.0])       |
|0    |Spark tiene varias varias varias herramientas|[spark, tiene, varias, varias, varias, herramientas]|(11,[0,1,2,4],[1.0,3.0,1.0,1.0])       |
|1    |Se analiza datos con herramientas            |[se, analiza, datos, con, herramientas]             |(11,[2,3,6,8,10],[1.0,1.0,1.0,1.0,1.0])|
+-----+---------------------------------------------+----------------------------------------------------+------------

In [7]:
# Aplicar TF-IDF (Inverse Document Frequency)
from pyspark.ml.feature import IDF

idf = IDF(inputCol='rawFeatures', outputCol='features')

# Podría aplicarse a df3 o df4, dado que ambos son TF
dfnew = idf.fit(df4).transform(df4)

# dfnew.columns
dfnew.select('label', 'features').show(truncate=False)

+-----+-------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                           |
+-----+-------------------------------------------------------------------------------------------------------------------+
|0    |(11,[0,5,7,9],[0.5753641449035617,0.6931471805599453,0.6931471805599453,0.6931471805599453])                       |
|0    |(11,[0,1,2,4],[0.28768207245178085,2.0794415416798357,0.28768207245178085,0.6931471805599453])                     |
|1    |(11,[2,3,6,8,10],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
+-----+-------------------------------------------------------------------------------------------------------------------+

